In [356]:
# Import Libraries and Dependencies

import os
import pandas as pd
#import hvplot.pandas
#import hvplot
import matplotlib.pyplot as plt
#import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
import warnings

from scipy.signal import argrelextrema

# Ignore any non-critical future warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [357]:
# Load .env file

load_dotenv()

True

In [358]:
# Set Alpaca API key and secret passwords

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [359]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [360]:
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis
# JAN 27 27 2021 GameStop event (GME)

start_date = pd.Timestamp("2021-01-27", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2021-1-28", tz="America/New_York").isoformat()

In [361]:
# Call stock data from identified period of time

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [362]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 191 entries, 2021-01-27 09:00:00+00:00 to 2021-01-28 00:55:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        191 non-null    float64
 1   high         191 non-null    float64
 2   low          191 non-null    float64
 3   trade_count  191 non-null    int64  
 4   open         191 non-null    float64
 5   volume       191 non-null    int64  
 6   vwap         191 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 11.9 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-01-27 09:00:00+00:00,330.42,330.58,329.93,43,329.93,10740,330.213696
2021-01-27 09:05:00+00:00,330.24,330.36,330.24,34,330.36,3193,330.313439
2021-01-27 09:10:00+00:00,330.21,330.24,330.18,14,330.18,1124,330.211967
2021-01-27 09:15:00+00:00,330.26,330.26,330.25,7,330.25,1360,330.254412
2021-01-27 09:20:00+00:00,330.13,330.25,330.13,10,330.23,1271,330.200189


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-01-28 00:35:00+00:00,317.14,317.14,316.87,96,316.87,28863,316.995793
2021-01-28 00:40:00+00:00,317.31,317.34,317.10,108,317.21,15182,317.154442
2021-01-28 00:45:00+00:00,317.29,317.47,317.21,108,317.47,18718,317.298227
2021-01-28 00:50:00+00:00,317.00,317.28,317.00,160,317.28,21358,317.156206
2021-01-28 00:55:00+00:00,317.20,317.24,316.99,133,316.99,21514,317.127020


In [363]:
# Clean up DF keep Open, High, Low, Close, Volume (OHLCV)

stock_data = stock_data.drop(columns=["trade_count", 'vwap'], axis=1)

stock_data.head()

,close,high,low,open,volume
timestamp,,,,,
2021-01-27 09:00:00+00:00,330.42,330.58,329.93,329.93,10740
2021-01-27 09:05:00+00:00,330.24,330.36,330.24,330.36,3193
2021-01-27 09:10:00+00:00,330.21,330.24,330.18,330.18,1124
2021-01-27 09:15:00+00:00,330.26,330.26,330.25,330.25,1360
2021-01-27 09:20:00+00:00,330.13,330.25,330.13,330.23,1271


In [377]:
# discrete dataset

x_data = stock_data.copy()     # the index will be our x axis, not date
y_data = stock_data.copy()

x_data.reset_index(inplace=True)
x_data = x_data.drop(columns=["timestamp"], axis=1)
x_data = x_data.index.tolist()

y_data.reset_index(inplace=True)
y_data = y_data.drop(columns=["timestamp"], axis=1)

#display(x_data.head())
display(y_data.head())

# x values for the polynomial fit, 200 points
#x = np.linspace(0, max(x_data.index.tolist()), max(x_data.index.tolist()))

# polynomial fit of degree xx
pol = np.polyfit(x_data, y_data, 17)
y_pol = np.polyval(pol, x)

# #           ___ plotting ___
# plt.figure(figsize=(15, 2), dpi= 120, facecolor='w', edgecolor='k')

# # plot stock data
# plt.plot(x_data, y_data, 'o', markersize=1.5, color='grey', alpha=0.7)

# # plot polynomial fit
# plt.plot(x, y_pol, '-', markersize=1.0, color='black', alpha=0.9)
# plt.legend(['stock data', 'polynomial fit'])
# plt.show()

,close,high,low,open,volume
0,330.42,330.58,329.93,329.93,10740
1,330.24,330.36,330.24,330.36,3193
2,330.21,330.24,330.18,330.18,1124
3,330.26,330.26,330.25,330.25,1360
4,330.13,330.25,330.13,330.23,1271


ValueError: operands could not be broadcast together with shapes (190,) (5,) 

In [ ]:
# ALPACAS DATA SMOOTHING/FIT

# def get_max_min(prices, smoothing, window_range):
#     smooth_prices = prices['close'].rolling(window=smoothing).mean().dropna()
#     local_max = argrelextrema(smooth_prices.values, np.greater)[0]
#     local_min = argrelextrema(smooth_prices.values, np.less)[0]
#     price_local_max_dt = []
#     for i in local_max:
#         if (i>window_range) and (i<len(prices)-window_range):
#             price_local_max_dt.append(prices.iloc[i-window_range:i+window_range]['close'].idxmax())
#     price_local_min_dt = []
#     for i in local_min:
#         if (i>window_range) and (i<len(prices)-window_range):
#             price_local_min_dt.append(prices.iloc[i-window_range:i+window_range]['close'].idxmin())  
#     maxima = pd.DataFrame(prices.loc[price_local_max_dt])
#     minima = pd.DataFrame(prices.loc[price_local_min_dt])
#     max_min = pd.concat([maxima, minima]).sort_index()
#     max_min.index.name = 'date'
#     max_min = max_min.reset_index()
#     max_min = max_min[~max_min.date.duplicated()]
#     p = prices.reset_index()   
#     max_min['day_num'] = p[p['timestamp'].isin(max_min.date)].index.values
#     max_min = max_min.set_index('day_num')['close']
    
#     return max_min

# smoothing = 1
# window_range = 10

# minmax = get_max_min(stock_data, smoothing, window)
# minmax


In [ ]:
# stock_data.reset_index()["close"].plot()
# plt.scatter(minmax.index, minmax.values, color="orange", alpha=.5)